In [1]:
import numpy as np, xarray as xr, matplotlib.pylab as plt, pandas as pd, seaborn as sns
import pickle, random, re, io, regionmask, dask, timeit, os, shutil, datetime
from scipy.signal import welch; from eofs.xarray import Eof; import matplotlib as mpl

from statsmodels.tsa.api import VAR; from contextlib import redirect_stdout
from distributed import Client; from scipy import stats
import cartopy.crs as ccrs, cartopy.feature as cfeature
import matplotlib.gridspec as gridspec; import cartopy.feature as cfeature
from matplotlib.colors import Normalize; import matplotlib.pyplot as plt
import warnings; warnings.filterwarnings('ignore')

from InVERT_functions import (calc_weights, concat_with_monthids, calc_EOFs, areaweighted_mean, 
autocorr, unstack_time, stack_time, createRandomSortedList, compare_T_pdfs, calc_psd_stats, 
plot_GMST_psd_spread, calc_emean_autocorrs, calc_ensemble_std_autocorrs, calc_efold_time, 
calc_eft_stats, compare_autocorrs_emean, plot_GMST_comparisons, save_region_means,
plot_regional_psd_spread, plot_regional_T_pdfs, plot_regional_emean_autocorrs, 
plot_regional_comparison, find_var_name, compare_MSE_to_emean_PSD, welch_psd, get_ensemble_variance, 
calc_emean_gridcell_MSE, plot_regional_variance_stats, plot_regional_eft_stats,
gridcell_map_plot, plot_gridcell_diff, plot_regional_diff_map, calc_gridcell_psd, plot_MSE_by_region,
calc_efold_time_dataset, plot_var_coeffs, convert_lon, emulate_pcs, plot_local_monthly_T_stds,
calc_gridcell_monthly_autocorrs)

In [2]:
scenario = 'Historical'

savepath = '/home/msaenger/InVERT/output/' # directory for saving InVERT output and training EOF data
    
lpath = '/home/msaenger/LENS2_' + scenario + '/TREFHT/' # directory where LENS2 training data is saved

In [3]:
n_samples = 25   # number of InVERT ensemble members to generate
n_steps = 1980   # number of time steps (months) to generate per InVERT ensemble member 
optimal_lag = 12 # VAR model lag (months)

nmodes = 100    # number of EOF modes to include
M = 120          # Number of initial time steps (months) to truncate for spin-up (aka 'burn-in' period)

In [4]:
LENS_esize = 50  # Size of LENS ensemble (number of members)

In [5]:
# Figure formatting
mpl.rcParams['font.family'] = 'sans-serif' # Change the default font family
tickfontsize = 14; axislabelfontsize=16
titlefontsize=18; legendfontsize=14
color1 = 'goldenrod'; color2='teal'

In [6]:
if scenario == 'Historical':
    scenario_name = 'HIST'
else: scenario_name = scenario

In [7]:
# #### DATA PREPROCESSING ####

# path = '/home/msaenger/CESM_LENS2/' + scenario + \
#        '/TREFHT/combined_by_ensemble_member/regridded/'

# ## combine 50 regridded ensemble member files into one dataset with 'ensemble' dimension
# ds = xr.open_mfdataset(path + '*.nc', concat_dim='ensemble', 
#                          combine='nested', parallel=True)

# ## assign ensemble coordinate
# ds = ds.assign_coords(ensemble=np.arange(len(ds['ensemble'])))

# ## Calculate ensemble mean and add to dataset as a variable
# ds['emean'] = ds.TREFHT.mean('ensemble')


# ## subtract ensemble mean to get anomalies 
# anoms = ds['TREFHT'] - ds['emean']

# anoms = anoms.to_dataset(name='anoms')
# anoms['gmean'] = areaweighted_mean(anoms.anoms)
# anoms.to_netcdf(lpath + 'LENS2_TREFHT_anomalies_regridded.nc')

In [8]:
anoms = xr.open_dataset(lpath + 'LENS2_TREFHT_anomalies_regridded.nc')

# anoms_concatted = concat_with_monthids(anoms)

# anoms_concatted['gmean'] = areaweighted_mean(anoms_concatted.anoms)

# ## Save month IDs from original T anomaly time series
# month_da = xr.DataArray(anoms_concatted.month.values,
#                         coords={'time': anoms_concatted.time.values, 
#                                 'month': ('time', anoms_concatted.month.values)},
#                         dims=['time'])

# anoms_concatted['month'] = month_da

# ## Save anomalies from ensemble mean that don't have monthly climatologies removed

# anoms_concatted.to_netcdf(lpath + 'LENS2_TREFHT_anomalies_regridded_concatted.nc')

In [9]:
## Randomly select a subset of ensemble members train on

n_training_members = 25 # Number of training ensemble members to use

In [10]:
training_members = (createRandomSortedList(n_training_members))

print('training members:', training_members)

training_anoms = anoms.sel(ensemble = [ens for ens in training_members])

training_anoms.to_netcdf(savepath + 'LENS2_Tanoms_' + str(n_training_members) + '_training_members.nc')

In [11]:
## Concatenate training ensemble members over time dimension and save

training_anoms_concatted = concat_with_monthids(training_anoms.anoms)

## Save month IDs from original T anomaly time series
month_da = xr.DataArray(training_anoms_concatted.month.values,
                        coords={'time': training_anoms_concatted.time.values, 
                                'month': ('time', training_anoms_concatted.month.values)},
                        dims=['time'])

training_anoms_concatted['month'] = month_da
training_anoms_concatted['gmean'] = areaweighted_mean(training_anoms_concatted.anoms)

training_anoms_concatted.to_netcdf(savepath + \
                        'LENS2_concatted_Tanoms_25_training_members.nc')

month_da.to_netcdf(savepath + 'month_da.nc')

In [12]:
## Save non-training ensemble members for diagnostics

testing_ensembles = [i for i in np.arange(0,50) if i not in training_members]

testing_anoms = anoms.sel(ensemble = random.sample(testing_ensembles, 25))

print('testing members:', testing_anoms.ensemble.values)
                         
testing_anoms.to_netcdf(savepath + 'LENS2_Tanoms_25_testing_members.nc')

In [13]:
## Concatenate non-training ensemble members over time dimension and save 

testing_anoms_concatted = concat_with_monthids(testing_anoms.anoms)

## Save month IDs from original T anomaly time series
month_da = xr.DataArray(testing_anoms_concatted.month.values,
                        coords={'time': testing_anoms_concatted.time.values, 
                                'month': ('time', testing_anoms_concatted.month.values)},
                        dims=['time'])

testing_anoms_concatted['month'] = month_da
testing_anoms_concatted['gmean'] = areaweighted_mean(testing_anoms_concatted.anoms)

testing_anoms_concatted.to_netcdf(savepath + \
            'LENS2_concatted_Tanoms_25_testing_members.nc')

In [14]:
### Calculate month-specific EOFs from training ensemble members

for month in range(1,13):

    month_EOFs = calc_EOFs(training_anoms_concatted.groupby('month')[month].anoms, path=savepath,
                      filename = 'LENS2_'+scenario_name+'_monthly_Tanom_EOFs_month='+str(month))
    print('month ' + str(month) + ' EOFs saved')

## Run emulator

In [15]:
# ## Load EOF solvers by month from training ensemble members

solvers_bymonth = {}
for month in range(1,13):
    solvers_bymonth[month] = calc_EOFs(0, path=savepath, filename='LENS2_'+scenario_name + \
                                       '_monthly_Tanom_EOFs_month='+str(month))

Loading EOF solver
Loading EOF solver
Loading EOF solver
Loading EOF solver
Loading EOF solver
Loading EOF solver
Loading EOF solver
Loading EOF solver
Loading EOF solver
Loading EOF solver
Loading EOF solver
Loading EOF solver


In [16]:
## Extract and save PCs, EOFs, and variance fractions from the EOF solver object

eofs_dict = {}
for month in range(1, 13):
    eofs_dict[month] = {}
    eofs_dict[month]['eofs'] = solvers_bymonth[month].eofs().sel(mode=slice(0, nmodes-1))
    eofs_dict[month]['pcs'] = solvers_bymonth[month].pcs().sel(mode=slice(0, nmodes-1))
    eofs_dict[month]['varfracs'] = solvers_bymonth[month].varianceFraction().sel(mode=slice(0, nmodes-1))
    
## Extract cos(lat) weights for later use

weights = solvers_bymonth[1].getWeights()
weights = xr.DataArray(weights, coords=[eofs_dict[1]['eofs']['lat'], 
                                        eofs_dict[1]['eofs']['lon']], 
                       dims=['lat', 'lon'])

In [18]:
### Compile DataArrays of PCs from the EOF solvers for each month

## Save PC data array (separated by ensemble member, e.g. 'unstacked') for each month and store in dictionary

pcs_unstacked = {}

for month in range(1, 13):

    pcs_unstacked[month] = unstack_time(eofs_dict[month]['pcs'].drop('month'), 
                                        esize = n_training_members) 
    
# Convert dictionary to xarray dataset. Variable names will be the month (1-12)

training_pcs_bymonth_unstacked = xr.Dataset(pcs_unstacked)

# Re-stack (e.g. concatenate) the training ensemble members over time

training_pcs_bymonth = stack_time(training_pcs_bymonth_unstacked)

In [20]:
# ## Extract each month's PC data array and adjust the 'time' values so as to put them 
# ## back together in time order (e.g. month 1 year 1, month 2 year 1, ... month 12 year 1, 
# ## month 1 year 2, month 2 year 2, ... etc)

month_pc_da_list = []

for month in range(1, 13):

    training_pcs_da_month = training_pcs_bymonth[month].drop('ensemble')
    training_pcs_da_month['time'] = training_pcs_da_month.time * 12 + month - 1
    training_pcs_da_month = training_pcs_da_month.to_dataset(name='pcs')
    month_pc_da_list.append(training_pcs_da_month)
    
# ## Merge into one dataset, sorted by time, and re-apply month coordinate

month_da = xr.open_dataarray(savepath + 'month_da.nc')

training_pcs = xr.merge(month_pc_da_list).sortby('time')
training_pcs['month'] = month_da.sel(time=slice(0,len(training_pcs['time'])))
training_pcs = training_pcs.assign_coords({'month': training_pcs.month})

# ## Save training PCs as netcdf 
training_pcs.to_netcdf(savepath + 'training_pcs.nc')

In [21]:
### Train 12 VAR models (one for each month of the year) using input-output pairs and standardization

### monthly_var_models will contain regression coefficients and residuals for each monthly VAR model

monthly_var_models = {}

for target_month in range(1, 13): # Iterate through each target month (1 for Jan to 12 for Dec)

    ## Lists to store input features and output targets for the current month 
    input_features = []; output_targets = []

    ## Iterate through the time dimension of the original training data to create input-output pairs
    ## Start the loop from optimal_lag so we have enough preceding data
    
    for i in range(optimal_lag, len(training_pcs.time)):
        
        ## Check if the current month is the target month
        if training_pcs.month.values[i] == target_month:
            
            ## Extract the preceding optimal_lag consecutive months of standardized PC data
            ## Flatten the data from (lag, mode) to a 1D array (lag * mode)
            features = training_pcs.pcs.values[i - optimal_lag : i, :].flatten()
            input_features.append(features)

            ## Extract the current month's PCs as the output target
            targets = training_pcs.pcs.values[i, :]
            output_targets.append(targets)

    ## Convert input and output feature lists to numpy arrays
    input_features = np.array(input_features); output_targets = np.array(output_targets)

    ## Check if enough data points to train the model
    if len(input_features) > 0:
        
        ## Fit a linear regression model using the input-output pairs
        ## Add a column of ones to input_features to account for the intercept (constant term)
        X = np.hstack([np.ones((input_features.shape[0],1)), input_features])
        y = output_targets

        ## Solve for the coefficients using least squares regression
        coefficients, residuals_info, rank, s = np.linalg.lstsq(X, y, rcond=None)

        ## Calculate the predicted values for the training data
        ## aka the linear combination (@) of the input features (lagged PCs and the intercept) 
            ## using the learned coefficients to produce the model's predicted PC values 
            ## for each instance of the target month in the training data
        predicted_targets = X @ coefficients

        ## Calculate the residuals (actual - predicted)
        ## shape is the number of target_months in the training_pc time series minus 1
            ## because the first instance of target_month won't have a full 12 months before it
        residuals = y - predicted_targets

        ## The first row of coefficients is the intercept of the lstsq calculation,
            ## the rest are for the lagged coefficients
        intercept = coefficients[0, :]
        lagged_coeffs = coefficients[1:, :]

        ## Store the VAR model components (coefficients, intercept, and residuals) for the target month
        ## Reshape lagged_coeffs back to (lag, input_mode, output_mode)
        monthly_var_models[target_month] = {
            'intercept': intercept,
            'lagged_coeffs': lagged_coeffs.reshape((optimal_lag, nmodes, nmodes)), 
            'residuals': residuals} 

In [22]:
## Create a list to store DataArrays for each month
monthly_coeffs_da_list = []

## Define coordinates
lags_coord = np.arange(optimal_lag, 0, -1) # Lags from 12 down to 1 (e.g. 'months ago')
input_modes_coord = np.arange(nmodes); output_modes_coord = np.arange(nmodes)

## Iterate through each month in the monthly_var_models dictionary
for month, components in monthly_var_models.items():
    lagged_coeffs = components['lagged_coeffs']

    ## Create a DataArray for the lagged coefficients of the current month
    coeffs_da = xr.DataArray(lagged_coeffs,
                             coords={'lag': lags_coord,
                                     'input_mode': input_modes_coord,
                                     'output_mode': output_modes_coord},
                             dims=['lag', 'input_mode', 'output_mode'])

    ## Add the month as a coordinate
    coeffs_da = coeffs_da.expand_dims(month=[month])

    monthly_coeffs_da_list.append(coeffs_da)

## Concatenate the DataArrays along the new 'month' dimension
lagged_coeffs_dataset = xr.concat(monthly_coeffs_da_list, dim='month')
lagged_coeffs_dataset = lagged_coeffs_dataset.to_dataset(name='lagged_coefficients')
lagged_coeffs_dataset.to_netcdf(savepath + 'lagged_coeff_dataset.nc')

In [23]:
## Plot VAR coefficients

plot_var_coeffs(lagged_coeffs_dataset, months_per_row=[1,2,3], savepath=savepath)
plot_var_coeffs(lagged_coeffs_dataset, months_per_row=[4,5,6], savepath=savepath)
plot_var_coeffs(lagged_coeffs_dataset, months_per_row=[7,8,9], savepath=savepath)
plot_var_coeffs(lagged_coeffs_dataset, months_per_row=[10,11,12], savepath=savepath)

In [24]:
## Emulate PCs and save as netcdf
InVERT_pcs = emulate_pcs(training_pcs, monthly_var_models, n_training_members,
                         optimal_lag, n_samples, n_steps, nmodes, M, savepath)

In [25]:
## Load emulated PCs
InVERT_pcs = xr.open_dataset(savepath + 'InVERT_PCs.nc')

In [26]:
## Separate PCs back into separate months, multiply by EOFs independently, and re-merge sorted by time

## Compute PCs * EOFs for each mode and divide by weights for every month. Save in dict.

print('Multiplying PCs * EOFs and dividing by weights')

products_by_month = {}

for month in range(1, 13):
    print(month)
    
    products_by_month[month] = InVERT_pcs.groupby('month')[month] * eofs_dict[month]['eofs'] / weights

In [27]:
## Sum T anomalies over modes then merge

print('Summing over modes')

products_by_month_summed = {}

for month in range(1, 13):
    
    print(month)
    
    products_by_month_summed[month] =  products_by_month[month].pcs.sum(dim='mode')

In [28]:
## Re-stack ensemble members over time dim (for easier re-sorting of months by time) and save in new dict

print('Stacking ensemble members over time')

Tanoms_bymonth = {}

for month in range(1, 13):
    print(month)
    
    Tanoms_bymonth[month] = (stack_time(products_by_month_summed[month]))

In [29]:
## Extract each month's T anomaly data array and adjust the 'time' values so as to put them 
## back together in time order (e.g. month 1 year 1, month 2 year 1, ... month 12 year 1, 
## month 1 year 2, month 2 year 2, ... etc)

print('Updating time indices')

Tanom_da_list = []

for month in range(1, 13):

    Tanoms_month = Tanoms_bymonth[month]
    Tanoms_month['time'] = Tanoms_month.time * 12 + month - 1
    Tanoms_month = Tanoms_month.to_dataset(name='T')
    Tanom_da_list.append(Tanoms_month)

In [30]:
## Concatenate over time dimension and then sort by time 
print('Merging and sorting by time')

InVERT_stacked = xr.concat(Tanom_da_list, dim='time').sortby('time')
InVERT_stacked['gmean'] = areaweighted_mean(InVERT_stacked.T)
InVERT_stacked.to_netcdf(savepath + 'InVERT_' + str(nmodes) + 'modes_stacked.nc')

In [31]:
## Separate into ensemble members

InVERT_T = unstack_time(InVERT_stacked, esize=n_samples)

print('Saving final InVERT dataset')

InVERT_T.to_netcdf(savepath + 'InVERT_'+str(nmodes)+'modes.nc')

print('saved')